# Day 26: Fairness Metrics (Fairlearn Principles)

In this lab, we will explore how to measure fairness in AI models using our `FairnessEvaluator`.
We will simulate a hiring model scenario where we want to ensure fairness across different demographic groups.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd

# Add root directory to sys.path
sys.path.append(os.path.abspath('../../'))

from src.fairness.metrics import FairnessEvaluator

## 1. Simulate Hiring Data

Let's create a synthetic dataset for 1000 applicants.
- **Y_true**: Whether the applicant was actually qualified (1) or not (0).
- **Group**: Sensitive attribute (e.g., 'Group A' vs 'Group B').
- **Scores**: The raw score output by our model.

In [ ]:
np.random.seed(42)
n = 1000

# Create groups (50/50 split)
groups = np.array(['Group A'] * 500 + ['Group B'] * 500)

# Ground truth: Both groups have similar qualification rates (~50%)
y_true = np.random.randint(0, 2, n)

# Model Bias Simulation: 
# The model gives Group A a boost in score, and Group B a penalty.
scores = np.random.rand(n)
scores[groups == 'Group A'] += 0.2
scores[groups == 'Group B'] -= 0.1

# Binary predictions with a threshold of 0.5
y_pred = (scores > 0.6).astype(int)

## 2. Evaluate Demographic Parity

Demographic Parity requires that the selection rate (percentage of applicants hired) is similar across groups.

In [ ]:
dp_diff = FairnessEvaluator.demographic_parity_difference(
    y_pred=y_pred,
    sensitive_features=groups
)

print(f"Demographic Parity Difference: {dp_diff:.4f}")

# Let's visualize the rates manually to confirm
df = pd.DataFrame({'Group': groups, 'Hired': y_pred})
print(df.groupby('Group').mean())

## 3. Evaluate Equalized Odds

Equalized Odds requires that True Positive Rates and False Positive Rates are similar across groups. This is a stricter metric that accounts for ground truth.

In [ ]:
eo_diff = FairnessEvaluator.equalized_odds_difference(
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=groups
)

print(f"Equalized Odds Difference: {eo_diff:.4f}")

## 4. Mitigate Bias (Naive Threshold Adjustment)

Let's try to fix this by lowering the threshold for Group B.

In [ ]:
# New predictions with group-specific thresholds
y_pred_mitigated = np.zeros(n)

# Group A threshold: 0.6
mask_a = (groups == 'Group A')
y_pred_mitigated[mask_a] = (scores[mask_a] > 0.6).astype(int)

# Group B threshold: 0.3 (Lowered to accept more)
mask_b = (groups == 'Group B')
y_pred_mitigated[mask_b] = (scores[mask_b] > 0.3).astype(int)

# Re-eval
dp_diff_new = FairnessEvaluator.demographic_parity_difference(
    y_pred=y_pred_mitigated,
    sensitive_features=groups
)

print(f"New Demographic Parity Difference: {dp_diff_new:.4f}")
df_new = pd.DataFrame({'Group': groups, 'Hired': y_pred_mitigated})
print(df_new.groupby('Group').mean())